In [14]:
from datasets import load_dataset

We use the mrpc dataset as an example dataset. It contains paraphrases and non-paraphrases. We will filter it and use only the paraphrases.

In [ ]:
raw_datasets = load_dataset('glue', 'mrpc')
raw_datasets

Let's store the splits separately and look at one example.

In [ ]:
raw_datasets_train = raw_datasets['train']
raw_datasets_val = raw_datasets['validation']
raw_datasets_test = raw_datasets['test']
raw_datasets_train[0]

Clearly label=1 means that the sentences are paraphrases. Let's filter them.

In [ ]:
ds_train = raw_datasets_train.filter(lambda x: x['label']==1)
ds_val = raw_datasets_val.filter(lambda x: x['label']==1)
ds_test = raw_datasets_test.filter(lambda x: x['label']==1)
len(ds_train), len(ds_val), len(ds_test)

Seems legit. That's a little over half the dataset.

Now, let's prepare the data for training.

In [18]:
from transformers import T5Tokenizer

In [19]:
checkpoint = 'google/mt5-small'
tokenizer = T5Tokenizer.from_pretrained(checkpoint)

FYI: [here](https://huggingface.co/learn/nlp-course/chapter7/4?fw=pt#processing-the-data) it says to specify `tokenizer.src_lang` and `tokenizer.tgt_lang` for multilingual models/tokeinzers, but this tokenizer does not have these properties.

Let's see how the tokenizer works:

In [ ]:
s1 = ds_train[0]['sentence1']
s2 = ds_train[0]['sentence2']
print(s1)
print(s2)
inputs = tokenizer(s1, text_target=s2)
inputs

Now we create a preprocess function that turns a dataset item into a form that the model can use for training.

First, let's find out what a reasonable `max_len` is.

In [ ]:
import matplotlib.pyplot as plt
train_lengths = list(map(lambda x: max(len(x['sentence1']), len(x['sentence2'])), ds_train))
val_lengths = list(map(lambda x: max(len(x['sentence1']), len(x['sentence2'])), ds_train))
test_lengths = list(map(lambda x: max(len(x['sentence1']), len(x['sentence2'])), ds_train))

plt.hist(train_lengths + val_lengths + test_lengths, 100)
plt.show()

Note that the y axis here is characters and not tokens. With 128 (for `max_len`) we are on the safe side.

In [42]:
max_length = 128

# the prefix has to (dynamically) be adjusted depending on the language or when training multilingually (I think).
prefix = 'paraphrase: '

def preprocess_function(examples):
    inputs = [prefix+s1 for s1 in examples['sentence1']]
    targets = examples['sentence2']
    # most likely there will be nothing to truncate, but we still add it
    model_inputs = tokenizer(inputs, text_target=targets, max_length=max_length, truncation=True)
    return model_inputs

Now we apply the preprocessing function to the datasets.

In [ ]:
tokenized_ds_train = ds_train.map(
    preprocess_function,
    batched=True,
    remove_columns=ds_train.column_names
)
tokenized_ds_val = ds_val.map(
    preprocess_function,
    batched=True,
    remove_columns=ds_val.column_names
)
tokenized_ds_test = ds_test.map(
    preprocess_function,
    batched=True,
    remove_columns=ds_test.column_names
)

Now the data is ready.

Next, the model and a Datacollator.

In [ ]:
from transformers import MT5ForConditionalGeneration

model = MT5ForConditionalGeneration.from_pretrained(checkpoint)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Here, I'll skip the example usage of the datacollator, check it out [here](https://huggingface.co/learn/nlp-course/chapter7/4?fw=pt#data-collation).

Now, let's continue with metrics. We will use Parascore.